In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import tensorflow as tf
from tensorflow import keras
import keras
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.layers import *
from tensorflow.keras.losses import MeanSquaredError

import numpy as np
import pandas as pd
import cvxpy as cp
from tqdm import tqdm

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
cwd = os.getcwd()

def make_dir(path):
    if os.path.exists(path) is False:
        os.makedirs(path)

def evaluate_prediction(predictions, actual, model_name):
    errors = predictions - actual
    mse = np.square(errors).mean()
    rmse = np.sqrt(mse)
    mae = np.abs(errors).mean()
    print('MAE: {:.2f}'.format(mae))
    print('RMSE: {:.2f}'.format(rmse))
    print('')
    print('')
    return mae, rmse

In [4]:
sl = pd.read_csv('dataset/ercot_sl_2019_2023.csv')
sl['sced_time_stamp_local'] = pd.to_datetime(sl['sced_time_stamp_local'])
sl.set_index('sced_time_stamp_local', inplace=True)
sl = sl.resample('h').mean()
date_range = pd.date_range(start=sl.index.min(), end=sl.index.max(), freq='h')
sl = sl[~sl.index.duplicated(keep='first')]
sl = sl.reindex(date_range, fill_value=np.nan)
sl.interpolate(method='time', inplace=True)


In [5]:
dap = pd.read_csv('dataset/hourly_ercot_day_ahead_sl_2019_2023.csv')
dap['timestamp'] = pd.to_datetime(dap['timestamp'])
dap.set_index('timestamp', inplace=True)
date_range = pd.date_range(start=dap.index.min(), end='2024-01-01 23:55:00', freq='h')
dap = dap[~dap.index.duplicated(keep='first')]
dap = dap.reindex(date_range, fill_value=np.nan)
dap.interpolate(method='time', inplace=True)

C:\Users\natha\AppData\Local\Temp\ipykernel_13976\3109986249.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dap['timestamp'] = pd.to_datetime(dap['timestamp'])


In [ ]:
forecasted_tlf = pd.read_csv('dataset/FORECASTED_TLF.csv')
forecasted_dlf = pd.read_csv('dataset/FORECASTED_DLF.csv')

In [7]:
#prepare the data for training and testing
df = pd.concat([dap, sl, forecasted_dlf, forecasted_tlf], axis=1)
df.columns = ['DAP', 'SCED', 'F_DLF', 'F_TLF']
df.dropna(inplace=True)

df

ValueError: Length mismatch: Expected axis has 6 elements, new values have 4 elements